Load cholec80 transforms and translate to different python module with same functionality. Also translate from yaml file to pandas pickle format

In [2]:
import sys
import copy
import pandas as pd

sys.path.append('../')

from utils import load_yaml, save_yaml

servers = load_yaml('../config/servers.yml')
server = servers['local']

# databases = load_yaml('../config/cholec80_transforms.yml')
# databases = load_yaml('../config/high_fps_without_camera_motion_videos_transforms.yml')
databases = load_yaml('../config/high_fps_with_camera_motion_videos_transforms.yml')
database = databases['databases'][0]

# files, prefix in db], transform {module, type, kwargs}, database {name, prefix}
database_df = pd.DataFrame(columns=['database', 'train', 'file', 'pre_transforms', 'aug_transforms', 'auxiliary'])

aug_transforms = [
    # {'module': 'torchvision.transforms', 'type': 'ConvertImageDtype', 'kwargs': {'dtype': torch.float32}},  # works, but kills worker (shared memory too small, might work on server) normalizes image already to [0., 1.]
    # {'module': 'torchvision.transforms', 'type': 'ColorJitter', 'kwargs': {'brightness': 0.5}}, # rework transforms here
    # {'module': 'torchvision.transforms', 'type': 'RandomGrayscale', 'kwargs': {'p': 0.5}},      # GaussianBlur, 
    # {'module': 'torchvision.transforms', 'type': 'RandomVerticalFlip', 'kwargs': {'p': 0.5}},   # RandomVerticalFlip & Horizontal -> to comp sq
    # {'module': 'torchvision.transforms', 'type': 'RandomHorizontalFlip', 'kwargs': {'p': 0.5}}  # RandomErasing, ConvertImageDtype
]

# load transforms and convert them to other module's callables
key_dict = {
    'Crop': 'Crop',
    'Resize': 'Resize'
}

for db_idx, db in enumerate(databases['databases']):

    for t_idx, old_transforms in enumerate(db['transforms']):
        pre_transforms = []
        for old_transform in old_transforms:
            # 'Crop' -> 'crop', shape -> height, width, top_left_corner -> top, left
            # 'Resize' -> 'resize', dsize -> size
            transform = {}
            for key, value in old_transform.items():
                if key == 'Crop':
                    transform = {
                        'module': 'utils.transforms',
                        'type': 'Crop',
                        'kwargs': {
                            'top_left_corner': value['top_left_corner'],
                            'shape': value['shape'],  # 'shape': value['shape'][::-1] 
                            'order': 'hwc'
                        }
                    }
                elif key == 'Resize':
                    transform = {
                        'module': 'utils.transforms',  # 'module': 'torchvision.transforms'
                        'type': 'Resize',
                        'kwargs': {
                            'dsize': value['dsize'],  # 'size': value[::-1]
                            'interpolation': 'cubic'
                        }
                    }
                    

                else:
                    raise ValueError('Key not known')

            pre_transforms.append(transform)   

        df_row = {
            'database': db['name'],
            'train': not db['test'],  # or put bool here
            'file': {'name': db['videos']['files'][t_idx], 'path': db['videos']['prefix']}, 
            'pre_transforms': pre_transforms,
            'aug_transforms': aug_transforms,
            'auxiliary': {}
        }
        database_df = database_df.append(df_row, ignore_index=True)

# database_df.to_pickle('../config/cholec80_transforms.pkl')
# database_df.to_pickle('../config/high_fps_without_camera_motion_videos_transforms.pkl')
database_df.to_pickle('../config/high_fps_with_camera_motion_videos_transforms.pkl')

print('pre-transforms:\n', database_df.pre_transforms[0])
print('aug-transforms:\n', database_df.aug_transforms[0])
database_df

pre-transforms:
 [{'module': 'utils.transforms', 'type': 'Crop', 'kwargs': {'top_left_corner': [18, 164], 'shape': [506, 632], 'order': 'hwc'}}, {'module': 'utils.transforms', 'type': 'Resize', 'kwargs': {'dsize': [816, 612], 'interpolation': 'cubic'}}]
aug-transforms:
 []


,database,train,file,pre_transforms,aug_transforms,auxiliary
0,SurgVisDom_test_no_indicators_without_camera_m...,True,"{'name': 'vid_4.mp4', 'path': 'without_camera_...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
1,SurgVisDom_test_no_indicators_without_camera_m...,True,"{'name': 'vid_5.mp4', 'path': 'without_camera_...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
2,SurgVisDom_test_no_indicators_without_camera_m...,True,"{'name': 'vid_6.mp4', 'path': 'without_camera_...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
3,SurgVisDom_test_no_indicators_without_camera_m...,True,"{'name': 'vid_9.mp4', 'path': 'without_camera_...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
4,SurgVisDom_test_no_indicators_without_camera_m...,True,"{'name': 'vid_10.mp4', 'path': 'without_camera...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
...,...,...,...,...,...,...
444,Mountney_without_camera_motion,True,"{'name': 'vid_444.avi', 'path': 'without_camer...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
445,Mountney_without_camera_motion,True,"{'name': 'vid_446.avi', 'path': 'without_camer...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
446,Mountney_without_camera_motion,False,"{'name': 'vid_445.avi', 'path': 'without_camer...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
447,Giannarou_without_camera_motion,True,"{'name': 'vid_448.avi', 'path': 'without_camer...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}


# Dummy Dataset for Testing

In [1]:
import pandas as pd
import torch

# create dummy df
database_df = pd.DataFrame(columns=['database', 'train', 'file', 'pre_transforms', 'aug_transforms', 'auxiliary'])

pre_transforms = [
    {'module': 'utils.transforms', 'type': 'Crop', 'kwargs': {'top_left_corner': [58, 142], 'shape': [554, 416], 'order': 'chw'}},
    {'module': 'torchvision.transforms', 'type': 'Resize', 'kwargs': {'size': [480, 640]}}
]

aug_transforms = [
    # {'module': 'torchvision.transforms', 'type': 'ConvertImageDtype', 'kwargs': {'dtype': torch.float32}},  # works, but kills worker (shared memory too small, might work on server) normalizes image already to [0., 1.]
    # {'module': 'torchvision.transforms', 'type': 'ColorJitter', 'kwargs': {'brightness': 0.1, 'contrast': 0.1, 'saturation': 0.1, 'hue': 0.1}}, # rework transforms here
    {'module': 'torchvision.transforms', 'type': 'RandomGrayscale', 'kwargs': {'p': 0.1}},      # GaussianBlur, 
    {'module': 'torchvision.transforms', 'type': 'RandomVerticalFlip', 'kwargs': {'p': 0.5}},    # RandomVerticalFlip & Horizontal
    {'module': 'torchvision.transforms', 'type': 'RandomHorizontalFlip', 'kwargs': {'p': 0.5}}    # RandomErasing, ConvertImageDtype
    # {'module': 'torchvision.transforms', 'type': 'GaussianBlur', 'kwargs': {'kernel_size': 9, 'sigma': (0.1, 1.)}}
]

for i in range(4):
    database_df = database_df.append(
        {
            'database': 'cholec80',
            'train': True,
            'file': {
                'name': 'video0{}_short.mp4'.format(i+1),
                'path': 'sample_videos'
            },
            'pre_transforms': pre_transforms,
            'aug_transforms': aug_transforms,
            'auxiliary': {}
        },
        ignore_index=True
    )

# add test set
for i in range(2):
    database_df = database_df.append(
        {
            'database': 'cholec80',
            'train': False,
            'file': {
                'name': 'video0{}_short.mp4'.format(i+1),
                'path': 'sample_videos'
            },
            'pre_transforms': pre_transforms,
            'aug_transforms': aug_transforms,
            'auxiliary': {}
        },
        ignore_index=True
    )


database_df.to_pickle('../config/cholec80_dummy_transforms.pkl')
print('pre-transforms:\n', database_df.pre_transforms[0])
print('aug-transforms:\n', database_df.aug_transforms[0])
database_df

pre-transforms:
 [{'module': 'utils.transforms', 'type': 'Crop', 'kwargs': {'top_left_corner': [58, 142], 'shape': [554, 416], 'order': 'chw'}}, {'module': 'torchvision.transforms', 'type': 'Resize', 'kwargs': {'size': [480, 640]}}]
aug-transforms:
 [{'module': 'torchvision.transforms', 'type': 'RandomGrayscale', 'kwargs': {'p': 0.1}}, {'module': 'torchvision.transforms', 'type': 'RandomVerticalFlip', 'kwargs': {'p': 0.5}}, {'module': 'torchvision.transforms', 'type': 'RandomHorizontalFlip', 'kwargs': {'p': 0.5}}]


,database,train,file,pre_transforms,aug_transforms,auxiliary
0,cholec80,True,"{'name': 'video01_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
1,cholec80,True,"{'name': 'video02_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
2,cholec80,True,"{'name': 'video03_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
3,cholec80,True,"{'name': 'video04_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
4,cholec80,False,"{'name': 'video01_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
5,cholec80,False,"{'name': 'video02_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}


In [15]:
import pandas as pd

# database_df = pd.read_pickle('../config/cholec80_transforms.pkl')
database_df = pd.read_pickle('../config/high_fps_without_camera_motion_videos_transforms.pkl')
# database_df = pd.read_pickle('../config/high_fps_with_camera_motion_videos_transforms.pkl')

print('pre-transforms:\n', database_df.pre_transforms[0])
print('aug-transforms:\n', database_df.aug_transforms[0])
database_df

pre-transforms:
 [{'module': 'utils.transforms', 'type': 'Crop', 'kwargs': {'top_left_corner': [18, 164], 'shape': [632, 506], 'order': 'chw'}}, {'module': 'torchvision.transforms', 'type': 'Resize', 'kwargs': {'size': [555, 740]}}]
aug-transforms:
 []


,database,train,file,pre_transforms,aug_transforms,auxiliary
0,SurgVisDom_test_no_indicators_without_camera_m...,True,"{'name': 'vid_4.mp4', 'path': 'without_camera_...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
1,SurgVisDom_test_no_indicators_without_camera_m...,True,"{'name': 'vid_5.mp4', 'path': 'without_camera_...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
2,SurgVisDom_test_no_indicators_without_camera_m...,True,"{'name': 'vid_6.mp4', 'path': 'without_camera_...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
3,SurgVisDom_test_no_indicators_without_camera_m...,True,"{'name': 'vid_9.mp4', 'path': 'without_camera_...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
4,SurgVisDom_test_no_indicators_without_camera_m...,True,"{'name': 'vid_10.mp4', 'path': 'without_camera...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
...,...,...,...,...,...,...
444,Mountney_without_camera_motion,True,"{'name': 'vid_444.avi', 'path': 'without_camer...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
445,Mountney_without_camera_motion,True,"{'name': 'vid_446.avi', 'path': 'without_camer...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
446,Mountney_without_camera_motion,False,"{'name': 'vid_445.avi', 'path': 'without_camer...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
447,Giannarou_without_camera_motion,True,"{'name': 'vid_448.avi', 'path': 'without_camer...","[{'module': 'utils.transforms', 'type': 'Crop'...",[],{}
